## Scrapping de informações disponibilizadas pela Fundação Getúlio Vargas
- A fundação Getúlio Vargas disponibiliza diversos dados para auxiliar na gestão de munícipios espalhados pelo Brasil;
- Porém, como se trata de uma ferramenta de consulta individual da situação de cada município, não é disponibilizada nenhuma ferramenta de extração de grandes massas de dados;
- Sendo assim o presente script foi desenvolvido para extrair dados de todos os municípios do Brasil de forma automatizada.

In [ ]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import itertools
import requests
import subprocess
import sqlite3

### Iniciando navegador e primeiras interações sob a página


In [ ]:
# Dicionário de elementos html e seus respectivos xpaths para 
# melhor organização das interações com a biblioteca Selenium

dicionario_html = {
                'botao_indicadores' : '/html/body/nav/div/div/ul/li[2]/a',
                'botao_municipios' : '/html/body/div[1]/div/div[2]/a[2]',
                'botao_saude' : '/html/body/div[2]/div[1]/div[2]/div/div/div/ul/li[6]/a',
                'botao_quant_UTI' : '/html/body/div[2]/div[2]/div/div/div/div[9]/div[3]/h2/button',
                'barra_pesq_muni' : '/html/body/div[1]/div[1]/div[3]/form/span/span[1]/span/span/textarea',
                'botao_deselec_muni' : '/html/body/div[1]/div[1]/div[3]/form/span/span[1]/span/ul/li/button/span',
                'botao_selec_muni' : '/html/body/span/span/span/ul/li/ul/li[1]',
                'corpo_grafico_quant_UTI' : '/html/body/div[2]/div[2]/div/div/div/div[9]/div[3]/div/div/div[3]/div/div/div[3]/div[1]/table/tbody'
                }

##### Tópico a serem extraídos da temática Saúde:
- Quantidade de médicos a cada 1000 habitantes
- Quantidade de leitos de UTI por município
- Gasto com saúde por habitante

In [ ]:
firefox_options = Options()
firefox_options.headless = True

navegador = webdriver.Firefox()
url = 'https://municipios.fgv.br/'
navegador.get(url)

wait = WebDriverWait(navegador, 10)
wait.until(EC.presence_of_element_located(('xpath', dicionario_html['botao_indicadores'])))
navegador.find_element('xpath', dicionario_html['botao_indicadores']).click()

wait.until(EC.presence_of_element_located(('xpath', dicionario_html['botao_municipios'])))
navegador.find_element('xpath', dicionario_html['botao_municipios']).click()

wait.until(EC.presence_of_element_located(('xpath', dicionario_html['botao_saude'])))
navegador.find_element('xpath', dicionario_html['botao_saude']).click()

# Quantidade de Leitos de UTI por município

#### Criando lista com o nome e sigla do estado de cada município do Brasil

In [ ]:
url_api = 'https://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Munic%C3%ADpios_com_IBGE_Brasil_Versao_CSV.csv'
response = requests.get(url_api)
dados = response.text

linhas = dados.strip().split('\n')
linhas_divididas = [linha.split(';') for linha in linhas]
df_cidades_estado = pd.DataFrame(linhas_divididas)
df_cidades_estado.columns = ['ConcatUF+Mun', 'IBGE', 'IBGE7', 'UF', 'Município', 'Região', 'População 2010', 'Porte', 'Capital', 'Extra', 'x', 'y', 'z']

cidades = []
estados = []
for i in range(len(df_cidades_estado)):
    cidades.append(df_cidades_estado['Município'].iloc[i])
    estados.append(df_cidades_estado['UF'].iloc[i])

cidades_estado = []
for i in range(len(cidades)):
    cidades_estado.append(cidades[i] + '/' + estados[i])

#### Tratamento na lista de cidades e estados

In [ ]:
resultado = []

index_principal = 0
for cidade_estado in cidades_estado:
    caracteres = list(cidade_estado)
    
    index_secundario = 0
    for caractere in caracteres:
        if caractere == '-' or caractere == "´":
            caracteres[index_secundario + 1] = caracteres[index_secundario + 1].upper()
        index_secundario += 1
        
    palavra_final = ''.join(caracteres)
    palavra_final = palavra_final.replace('´', "'")
    cidades_estado[index_principal] = palavra_final
    index_principal += 1

cidades_estado.pop(0)



#### COPIANDO OS MUNICIPIOS TRATADOS

In [6]:

resultado = ' '.join(cidades_estado)

# Copia o resultado para a área de transferência
def area_transferencia(resultado):

    try:
        processo = subprocess.Popen('clip', env={'LANG': 'pt_BR.UTF-16'}, stdin=subprocess.PIPE)
        processo.communicate(resultado.encode('utf-16'))
        resultado
    except Exception as e:
        print("Erro ao copiar para a área de transferência:", str(e))
        
def formatar_cidade_estado(cidade_estado):
    municipio, estado = cidade_estado.split('/')
    municipio = municipio.strip().replace('-', ' ').title().replace('´', "'").replace('`', "'")
    estado = estado.strip().lower()
    return f"{municipio}/{estado}"

#Criando conexao SQL
conn = sqlite3.connect('municipios.db')

#cursor SQL
cursor = conn.cursor()

# Verificando se a tabela já existe
cursor.execute("SELECT count(name) FROM sqlite_master WHERE type='table' AND name='municipios'")
if cursor.fetchone()[0] == 0:

#Criando tabela
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS municipios (
        id INTEGER PRIMARY KEY,
        municipio TEXT,
        uf TEXT
    )
''')

for cidade_estado in cidades_estado:
    resultado_formatado = formatar_cidade_estado(cidade_estado)
    area_transferencia(resultado_formatado)
    # Aguarde uma entrada do usuário antes de prosseguir para o próximo
    input("Pressione Enter para copiar o próximo município/UF...")
    
conn.close()
Rotina = ("Tabela criada e preenchida e item copiado")

Rotina

#### Iterando Sob a lista de cidades e estados e extraindo o copo HTML necessário


#### Extraindo informações da quantidade de leitos de UTI do corpo HTML da tabela